Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1.1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

** Add L2 regularization for logistic regression model first. Note: L2 is a penalty to the Loss function.

In [25]:
batch_size = 128
LR = 1e-1
L2_weight = 0.5e-3
graph = tf.Graph()

def apply_l2_norm(l2_weight, weights):
    return l2_weight * tf.nn.l2_loss(weights)

def build_loss_with_l2_norm(logits, train_labels, weights):
#     + apply_l2_norm(L2_weight, weights)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = train_labels))

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = build_loss_with_l2_norm(logits, tf_train_labels, weights)
    
    optimizer = tf.train.GradientDescentOptimizer(LR).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [26]:
num_steps = 9001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step: 0: 20.485775
Minibatch accuracy: 9.4%
Validation dataset accuracy: 10.5%
Minibatch loss at step: 500: 2.165907
Minibatch accuracy: 68.0%
Validation dataset accuracy: 71.4%
Minibatch loss at step: 1000: 2.246017
Minibatch accuracy: 71.1%
Validation dataset accuracy: 73.8%
Minibatch loss at step: 1500: 1.350790
Minibatch accuracy: 81.2%
Validation dataset accuracy: 74.9%
Minibatch loss at step: 2000: 1.279726
Minibatch accuracy: 78.1%
Validation dataset accuracy: 75.2%
Minibatch loss at step: 2500: 1.697566
Minibatch accuracy: 69.5%
Validation dataset accuracy: 75.9%
Minibatch loss at step: 3000: 1.640931
Minibatch accuracy: 71.9%
Validation dataset accuracy: 76.1%
Minibatch loss at step: 3500: 1.499397
Minibatch accuracy: 73.4%
Validation dataset accuracy: 76.3%
Minibatch loss at step: 4000: 1.079835
Minibatch accuracy: 76.6%
Validation dataset accuracy: 76.5%
Minibatch loss at 

**Learning rate: 0.1, batch_size: 128, steps: 9001(5.7 epoch), Test accuracy: 84.0%.**

---
Problem 1.2
---------
Add L2 regularization for one hidden layer fully connected network.

In [28]:
batch_size = 128
relu_count = 1024
L2_weight = 0.5e-3

graph_nn_l2 = tf.Graph()
with graph_nn_l2.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_hidden_layer = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
    biases_hidden_layer = tf.Variable(tf.zeros(relu_count))
    
    weights_output_layer = tf.Variable(tf.truncated_normal([relu_count, num_labels]))
    biases_output_layer = tf.Variable(tf.zeros(num_labels))
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_layer) + biases_hidden_layer)
    
    logits = tf.matmul(hidden_layer_output, weights_output_layer) + biases_output_layer
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))\
    + L2_weight * (tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(weights_output_layer))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer) 
        + biases_output_layer)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer)
        + biases_output_layer)

In [30]:
num_steps = 6001

with tf.Session(graph=graph_nn_l2) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step: 0: 465.564636
Minibatch accuracy: 11.7%
Validation dataset accuracy: 23.8%
Minibatch loss at step: 500: 128.918060
Minibatch accuracy: 83.6%
Validation dataset accuracy: 79.7%
Minibatch loss at step: 1000: 98.879898
Minibatch accuracy: 76.6%
Validation dataset accuracy: 80.3%
Minibatch loss at step: 1500: 73.096237
Minibatch accuracy: 90.6%
Validation dataset accuracy: 81.7%
Minibatch loss at step: 2000: 56.606525
Minibatch accuracy: 84.4%
Validation dataset accuracy: 82.4%
Minibatch loss at step: 2500: 43.671825
Minibatch accuracy: 83.6%
Validation dataset accuracy: 83.1%
Minibatch loss at step: 3000: 33.988770
Minibatch accuracy: 84.4%
Validation dataset accuracy: 85.0%
Minibatch loss at step: 3500: 26.555674
Minibatch accuracy: 86.7%
Validation dataset accuracy: 85.8%
Minibatch loss at step: 4000: 20.655241
Minibatch accuracy: 90.6%
Validation dataset accuracy: 86.1%
Minibat

Test accuracy: 93.8%, so far, so good.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [31]:
num_steps = 3001
small_batch_data = train_dataset[:batch_size * 7, :]
small_batch_labels = train_labels[:batch_size * 7]
with tf.Session(graph=graph_nn_l2) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (small_batch_data.shape[0] - batch_size)
        
        batch_data = small_batch_data[offset:(offset + batch_size), :]
        batch_labels = small_batch_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step: 0: 520.370361
Minibatch accuracy: 13.3%
Validation dataset accuracy: 30.5%
Minibatch loss at step: 500: 122.112846
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.9%
Minibatch loss at step: 1000: 95.098579
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.8%
Minibatch loss at step: 1500: 74.060570
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.9%
Minibatch loss at step: 2000: 57.676636
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.9%
Minibatch loss at step: 2500: 44.917229
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.9%
Minibatch loss at step: 3000: 34.980492
Minibatch accuracy: 100.0%
Validation dataset accuracy: 76.0%
Test dataset accuracy: 83.2%


Training dataset accuracy reaches 100% very quickly, and the final test dataset accuracy is, in fact, not as bad as I expect.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

** Apply drop out to one hidden layer neural network. Too see if it's better than L2 norm.

In [32]:
batch_size = 128
relu_count = 1024
L2_weight = 0.5e-3

graph_nn_dropout = tf.Graph()
with graph_nn_dropout.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_hidden_layer = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
    biases_hidden_layer = tf.Variable(tf.zeros(relu_count))
    
    weights_output_layer = tf.Variable(tf.truncated_normal([relu_count, num_labels]))
    biases_output_layer = tf.Variable(tf.zeros(num_labels))
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_layer) + biases_hidden_layer)
    
#   apply drop out.
    dropout_probablity = tf.constant(0.5)
    output_layer_activition = tf.nn.dropout(hidden_layer_output, dropout_probablity)
    
    logits = tf.matmul(output_layer_activition, weights_output_layer) + biases_output_layer\
    + L2_weight * (tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(weights_output_layer))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5e-2).minimize(loss)
    
    train_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_train_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer) 
        + biases_output_layer)
    
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer) 
        + biases_output_layer)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer)
        + biases_output_layer)

In [ ]:
num_steps = 6001
with tf.Session(graph=graph_nn_dropout) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step: 0: 501.612396
Minibatch accuracy: 9.4%
Validation dataset accuracy: 9.0%


1. Hidden layer neurons: 1024, learning rate: 0.5, num_steps: 3001. With dropout only, Test accuracy: 87.5%
2. Hidden layer neurons: 1024, learning rate: 0.5, num_steps: 3001. dropout + L2 norm, Test accuracy: 87.2%
3. Hidden layer neurons: 1024, learning rate: 0.05, num_steps: 3001. dropout + L2 norm, Test accuracy: 90.6%
4. 3. Hidden layer neurons: 1024, learning rate: 0.05, num_steps: 6001. dropout + L2 norm, Test accuracy: 90.6%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [26]:
batch_size = 128
image_size = 28
num_labels = 10
layer_1_relu_counts = 512
layer_2_relu_counts = 256
L2_weight = 0.5e-3
LR = 1e-3

def get_scope_variable(scope_name, variable_name, shape, initializer):
    with tf.variable_scope(scope_name) as scope:
        try:
            variable = tf.get_variable(variable_name, shape, initializer = initializer)
            return variable
        except ValueError:
            scope.reuse_variables()
            variable = tf.get_variable(variable_name, shape)
            return variable

def fc_nn_model_with_multi_layer(tf_train_data, image_size, layer_1_relu_counts, layer_2_relu_counts, num_labels):
    weights = get_scope_variable("hidden_layer_1", "weights", [image_size * image_size, layer_1_relu_counts], initializer = tf.truncated_normal_initializer())
    biases = get_scope_variable("hidden_layer_1", "biases", [layer_1_relu_counts], initializer = tf.zeros_initializer())
    
    l2_norm_1 = tf.nn.l2_loss(weights)
    
    relu_1 = tf.nn.relu(tf.matmul(tf_train_data, weights) + biases)
        
    weights = get_scope_variable("hidden_layer_2", "weights", [layer_1_relu_counts, layer_2_relu_counts], initializer = tf.truncated_normal_initializer())
    biases = get_scope_variable("hidden_layer_2", "biases", [layer_2_relu_counts], initializer = tf.zeros_initializer())
    
    l2_norm_2 = tf.nn.l2_loss(weights)
    
    relu_2 = tf.nn.relu(tf.matmul(relu_1, weights) + biases)
        
    weights = get_scope_variable("output_layer", "weights", [layer_2_relu_counts, num_labels], initializer = tf.truncated_normal_initializer())
    biases = get_scope_variable("output_layer", "biases", [num_labels], initializer = tf.zeros_initializer())
    
    l2_norm_3 = tf.nn.l2_loss(weights)
    
    logits = (tf.matmul(relu_2, weights) + biases) + L2_weight * (l2_norm_1 + l2_norm_2 + l2_norm_3)
    
    return logits

def build_loss(logits, tf_train_labels):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    return loss

def get_learning_rate():
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(learning_rate = LR, global_step = global_step, decay_steps = 500, decay_rate = 0.90)
    return global_step, learning_rate

def evaluate_model(data, image_size, layer_1_relu_counts, layer_2_relu_counts, num_labels):
    logits = fc_nn_model_with_multi_layer(data, image_size, layer_1_relu_counts, layer_2_relu_counts, num_labels)
    prediction = tf.nn.softmax(logits)
    return prediction

multi_layer_nn_graph = tf.Graph()
with multi_layer_nn_graph.as_default():
    tf_train_data = tf.placeholder(tf.float32, shape = [batch_size, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    logits = fc_nn_model_with_multi_layer(tf_train_data, image_size, layer_1_relu_counts, layer_2_relu_counts, num_labels)
    
    loss = build_lostt_with_l2_norm(logits, tf_train_labels, L2_weight, image_size, layer_1_relu_counts, layer_2_relu_counts, num_labels)
    
    global_step, learning_rate = get_learning_rate()
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = evaluate_model(tf_valid_dataset, image_size, layer_1_relu_counts, layer_2_relu_counts, num_labels)
    test_prediction = evaluate_model(tf_test_dataset, image_size, layer_1_relu_counts, layer_2_relu_counts, num_labels)

In [27]:
num_steps = 18001
with tf.Session(graph = multi_layer_nn_graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized\n")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset: (offset+batch_size), :]
        batch_labels = train_labels[offset: (offset+batch_size)]
        
        feed_dict = {tf_train_data: batch_data, tf_train_labels: batch_labels}
        
        _, loss_eval, train_prediction_eval = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, loss_eval))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction_eval, batch_labels))

    test_prediction_eval = session.run(test_prediction)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_eval, test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized

Minibatch loss at step: 0: 2221.656006
Minibatch accuracy: 16.4%
Minibatch loss at step: 500: 142.417206
Minibatch accuracy: 78.9%
Minibatch loss at step: 1000: 181.085846
Minibatch accuracy: 73.4%
Minibatch loss at step: 1500: 87.866066
Minibatch accuracy: 81.2%
Minibatch loss at step: 2000: 88.283707
Minibatch accuracy: 80.5%
Minibatch loss at step: 2500: 75.269264
Minibatch accuracy: 82.0%
Minibatch loss at step: 3000: 86.469498
Minibatch accuracy: 75.0%
Minibatch loss at step: 3500: 66.399673
Minibatch accuracy: 82.8%
Minibatch loss at step: 4000: 45.375114
Minibatch accuracy: 80.5%
Minibatch loss at step: 4500: 53.159958
Minibatch accuracy: 78.1%
Minibatch loss at step: 5000: 45.129204
Minibatch accuracy: 82.8%
Minibatch loss at step: 5500: 58.595623
Minibatch accuracy: 78.1%
Minibatch loss at step: 6000: 61.892845
Minibatch accuracy: 78.1%
Minibatch loss at step: 6500: 45.255348
Minibatch accur